<h1><center>In-Depth EDA [All Files]+Competition Understanding</center></h1>
<h3><center>H&M Personalized Fashion Recommendations</center></h3>

<center><img src = "http://www.hm.com/entrance/assets/bundle/img/HM-Share-Image.jpg" width = "750" height = "500"/></center>                                                                          

<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Contents</center></h2>

> | S.No       |                   Heading                |
> | :------------- | :-------------------:                |         
> |  01 |  [**Competition Overview**](#competition-overview)  |                   
> |  02 |  [**Libraries**](#libraries)                        |  
> |  03 |  [**Global Config**](#global-config)                |
> |  04 |  [**Weights and Biases**](#weights-and-biases)      |
> |  05 |  [**Load Datasets**](#load-datasets)                |
> |  06 |  [**Articles EDA**](#articles-eda)  |

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='background:maroon; border:0; color:white' role="tab" aria-controls="home"><center>If you find this notebook useful, do give me an upvote, it helps to keep up my motivation. This notebook will be updated frequently so keep checking for furthur developments.</center></h3>

---

<a id="competition-overview"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Competition Overview</center></h2>

## **<span style="color:orange;">Description</span>**
  
In this competition, H&M Group invites you to develop product recommendations based on data from previous transactions, as well as from customer and product meta data. The available meta data spans from simple data, such as garment type and customer age, to text data from product descriptions, to image data from garment images.
  
There are no preconceptions on what information that may be useful – that is for you to find out. If you want to investigate a categorical data type algorithm, or dive into NLP and image processing deep learning, that is up to you.

---

## **<span style="color:orange;">Evaluation Metric</span>**

Submissions are evaluated according to the Mean Average Precision @ 12 (MAP@12).

**Notes:**

- You will be making purchase predictions for all customer_id values provided, regardless of whether these customers made purchases in the training data.
- Customer that did not make any purchase during test period are excluded from the scoring.
- There is never a penalty for using the full 12 predictions for a customer that ordered fewer than 12 items; thus, it's advantageous to make 12 predictions for each customer.

---

<a id="libraries"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Libraries</center></h2>

In [ ]:
# Necessities
import os
import random

import numpy as np
import pandas as pd

#Text Processing
import re
import nltk
nltk.download('popular')

# Aesthetics
from termcolor import colored

# Data Visualization
import plotly.express as px

import matplotlib.pyplot as plt
%matplotlib inline

<a id="global-config"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Global Config</center></h2>

In [ ]:
class config:
    SEED=42
    DIRECTORY_PATH = "../input/h-and-m-personalized-fashion-recommendations"
    IMAGES_FILE_PATH = os.path.join(DIRECTORY_PATH, 'images')
    ARTICLES_FILE_PATH = os.path.join(DIRECTORY_PATH, 'articles.csv')
    CUSTOMERS_FILE_PATH = os.path.join(DIRECTORY_PATH, 'customers.csv')
    SAMPLE_FILE_PATH = os.path.join(DIRECTORY_PATH, 'sample_submission.csv')
    TRANSACTIONS_FILE_PATH = os.path.join(DIRECTORY_PATH, 'transactions_train.csv')

In [ ]:
def set_seed(seed=config.SEED):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    
set_seed()

---

<a id="weights-and-biases"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Weights and Biases</center></h2>

<center><img src = "https://i.imgur.com/1sm6x8P.png" width = "750" height = "500"/></center>  

**Weights & Biases** is the machine learning platform for developers to build better models faster.

You can use W&B's lightweight, interoperable tools to

- quickly track experiments,
- version and iterate on datasets,
- evaluate model performance,
- reproduce models,
- visualize results and spot regressions,
- and share findings with colleagues.
  
Set up W&B in 5 minutes, then quickly iterate on your machine learning pipeline with the confidence that your datasets and models are tracked and versioned in a reliable system of record.

In this notebook I will use Weights and Biases's amazing features to perform wonderful visualizations and logging seamlessly.

---

<a id="load-datasets"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Load Datasets</center></h2>

In [ ]:
articles = pd.read_csv(config.ARTICLES_FILE_PATH)
customers = pd.read_csv(config.CUSTOMERS_FILE_PATH)
sample = pd.read_csv(config.SAMPLE_FILE_PATH)
# transactions = pd.read_csv(config.TRANSACTIONS_FILE_PATH)

<a id="articles-eda"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Articles EDA</center></h2>

## **<span style="color:orange;">Basic Tabular Analysis</span>**

### Head

In [ ]:
articles.head()

### Dataset Size

In [ ]:
print("Shape of articles.csv: ", colored(articles.shape, 'yellow'))

Thus, the file has 105,542 rows and 25 columns.

### Info

In [ ]:
articles.info()

This tells us that there are no null values in the **articles.csv** file.

### Column wise Unique Values

In [ ]:
for col in articles.columns:
    print(col + ":" + colored(str(len(articles[col].unique())), 'yellow'))

---

## **<span style="color:orange;">Unique Value Counts</span>**

### Product Name

In [ ]:
articles.prod_name.value_counts()

### Product Type Name

In [ ]:
articles.product_type_name.value_counts()

### Product Group Name

In [ ]:
articles.product_group_name.value_counts()

### Graphical Appearance Name

In [ ]:
articles.graphical_appearance_name	.value_counts()

### Colour Group Name

In [ ]:
articles.colour_group_name.value_counts()

---

## **<span style="color:orange;">Detail Description Distributions</span>**

In [ ]:
articles.detail_desc

### Preprocessing the text

In [ ]:
def preprocess_text(text, flg_stemm=False, flg_lemm=True):

    lst_stopwords = nltk.corpus.stopwords.words("english")
    
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()    
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text


### Clean the text

In [ ]:
# Clean Detail Description
articles["clean_detail_desc"] = articles["detail_desc"].apply(
    lambda x: preprocess_text(x, flg_stemm=False, flg_lemm=True, )
)

### Generate New Features

In [ ]:
#Length of Detail Description
articles['clean_detail_desc_len'] = articles['clean_detail_desc'].apply(lambda x: len(x))

#Word Count
articles['clean_detail_desc_word_count'] = articles["clean_detail_desc"].apply(lambda x: len(str(x).split(" ")))

#Character Count
articles['clean_detail_desc_char_count'] = articles["clean_detail_desc"].apply(lambda x: sum(len(word) for word in str(x).split(" ")))

#Average Word Length
articles['clean_detail_desc_avg_word_length'] = articles['clean_detail_desc_char_count'] / articles['clean_detail_desc_word_count']

### Distribution Plots

In [ ]:
def plot_distribution(x, title):

    fig = px.histogram(
    articles, 
    x = x,
    width = 800,
    height = 500,
    title = title
    )
    
    fig.show()

In [ ]:
plot_distribution(x = 'clean_detail_desc_len', title = 'Description Length Distribution')

In [ ]:
plot_distribution(x = 'clean_detail_desc_word_count', title = 'Word Count Distribution')

In [ ]:
plot_distribution(x = 'clean_detail_desc_char_count', title = 'Character Count Distribution')

In [ ]:
plot_distribution(x = 'clean_detail_desc_avg_word_length', title = 'Average Word Length Distribution')

---